# Stock market analysis during the Covid-19 pandemic

#### Summary:
The Covid-19 pandemic had severe impacts on stock markets. It would be informative to identify the trends of a chosen industry and validate the claims/forecasts of different analysts and agencies. Through public data from yahoo finance, stock market data can be obtained to identify these trends. Also, through qualitative research on various agencies such as Morningstar, their claims can be validated with data. An overall stock price trend report of a chosen industry can be used as a general guide for future pandemics and financial crises.

## Libraries

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

## Validate analyst claims 

### Morningstar
Ali Mogharabi from Morningstar: 'both companies (Uber and Lyft) will recover from this, although it will take until next year.’

Prediction: Recovery will take until 2021, where recovery is taken to be pre-covid levels (defined as highest price of stock between Jan-Feb 2020 which was just before Covid-19)

In [16]:
#Markets open on 2nd Jan as 1st Jan is public holiday
#Time horizon from Jan 2020 - Apr 2021
data = yf.download("UBER LYFT", start="2020-01-02", end="2021-04-30", interval="1d", group_by="ticker")
data = data.reset_index() #to convert date from index to column 

[*********************100%***********************]  2 of 2 completed


In [76]:
data.head()

Date       UBER                                                        \
                   Open       High        Low      Close  Adj Close    Volume   
0 2020-01-02  29.940001  31.000000  29.790001  30.990000  30.990000  20578900   
1 2020-01-03  30.620001  31.430000  30.480000  31.370001  31.370001  18822700   
2 2020-01-06  31.010000  32.060001  31.000000  31.580000  31.580000  21204700   
3 2020-01-07  31.790001  32.840000  31.360001  32.810001  32.810001  30119600   
4 2020-01-08  32.730000  34.520000  32.459999  33.930000  33.930000  43944400   

        LYFT                                                       
        Open       High        Low      Close  Adj Close   Volume  
0  43.220001  44.070000  43.165001  43.580002  43.580002  3912100  
1  42.849998  43.360001  42.233002  43.189999  43.189999  4407200  
2  42.790001  43.810001  42.380001  43.099998  43.099998  4964600  
3  43.180000  44.700001  43.029999  44.250000  44.250000  4773500  
4  44.139999  45.740002  43.869999  45.080002  45.080002  5636300

In [137]:
#empty dataframe to store results
col = ['Stock', 'Pre-Covid level', 'Earliest recovery date', 'Price']
recover_df = pd.DataFrame(columns = col)

#function that takes in stock ticker symbol and returns pre covid stock price which is defined as the highest price of the stock
#between Jan-Feb 2020, the earliest date that the stock hits or surpasses pre-covid level and the price. 
def recover(stock):
    index = int(data[data['Date'] == '2020-02-28'].index.values) #index for last date of trading in feb 2020
    pre_covid = round(max(data[stock]['Close'].iloc[:index]),2)
    for i in range(index+1,len(data)):
        if data[stock]['Close'].iloc[i]>= pre_covid:
            date = data['Date'].iloc[i].date()
            price = round(data[stock]['Close'].iloc[i],2)
            break
    df = pd.Series([stock,pre_covid,date,price],index = col)
    global recover_df
    recover_df = recover_df.append(df, ignore_index = True)
    
for i in ['UBER', 'LYFT']:
    recover(i)
recover_df



,Stock,Pre-Covid level,Earliest recovery date,Price
0,UBER,41.27,2020-11-05,41.96
1,LYFT,53.94,2021-02-10,56.21


Prediction that recovery will take until 2021 is partially true. Actual recovery was quicker for Uber that occurred on 5 Nov 2020, but actual recovery was as predicted for Lyft that occurred on 10 Feb 2021 when stock prices recovered to pre-covid levels.

## Fair value estimate
Morningstar provides fair value estimates for stocks which are like reference points that an investor can use to determine if a stock is undervalued or overvalued. 

In [188]:
#empty dataframe to store results
col = ['Stock', 'Current date', 'Current price', 'Fair value estimate', 'Actual price', 'Time to reach fair/max/min value', 
       'Profit','Hit fair value?']
fairvalue_df = pd.DataFrame(columns = col)

#function that takes in the stock ticker symbol, fair value estimate, date of analysis,
#and returns when the fair value estimate is achieved, the profit earned from buying the stock when price < fair value, 
#and if price increases further when price > fair value 
def fairvalue(stock, estimate, date):
    data = yf.download(stock, start="2020-01-02", end="2021-04-30", interval="1d")
    data = data.reset_index()
    index = data[data['Date'] == date].index.values
    price = np.round(data['Close'].iloc[index].values[0],2)
    if price < estimate: # buy since price < fair value
        if max(data['Close'].iloc[int(index):]) < estimate: # did not hit fair value
            actualprice = round(max(data['Close'].iloc[int(index):]),2)
            maxprice = data[data['Close'] == max(data['Close'].iloc[int(index):])].index.values
            time = [data['Date'].iloc[int(maxprice)].date(), data['Date'].iloc[int(maxprice)] - pd.to_datetime(date) ]
            profit = "{:.0%}".format((actualprice-price)/price)
        else: # hit fair value
            for i in range(int(index),len(data)):
                if data['Close'][i] >= estimate:
                    actualprice = round(data['Close'].iloc[i],2)
                    time = [data['Date'].iloc[i].date(),data['Date'].iloc[i]-pd.to_datetime(date)]
                    profit = "{:.0%}".format((actualprice-price)/price)
                    break
        if actualprice < estimate:
            hit = 'No'
        else:
            hit = 'Yes'
    else: # sell since price > fair value
        for i in range(int(index),len(data)):
            if data['Close'][i] <= estimate: # hit fair value
                actualprice = round(data['Close'][i],2)
                time = [data['Date'][i].date(),data['Date'][i]-pd.to_datetime(date)]
                profit = "{:.0%}".format((price - actualprice)/actualprice)
                break
            else: # did not hit fair value
                actualprice = round(min(data['Close'].iloc[int(index):]),2)
                minprice = data[data['Close'] == min(data['Close'].iloc[int(index):])].index.values
                time = [data['Date'].iloc[int(minprice)].date(), data['Date'].iloc[int(minprice)] - pd.to_datetime(date) ]
                profit = "{:.0%}".format((price - actualprice)/actualprice)
                break
        if actualprice <= estimate:
            hit = 'Yes'
        else:
            hit = 'No'
    df = pd.Series([stock,date,'$'+str(price),'$'+str(estimate),'$'+str(actualprice),time,profit,hit], index = col)
    global fairvalue_df
    fairvalue_df = fairvalue_df.append(df, ignore_index = True)
    
    
    
    

In [189]:
#analysis from https://www.morningstar.com/articles/972772/5-stocks-we-like
#buy as fair value > current price
stocks = ['FANG', 'FSLR', 'SYY', 'CMCSA']
date = '2020-03-17'
fair_value = [90,59,59,49]
#function that takes in stock ticker, date, fair value and returns output of fairvalue function defined above
def morningstar(stocks, date, fair_value):
    stock_dict = {'stocks': stocks, 'date': date, 'fair_value': fair_value}
    for i in range(len(stock_dict['stocks'])):
        stock = stock_dict['stocks'][i]
        estimate = stock_dict['fair_value'][i]
        fairvalue(stock,estimate,date)
morningstar(stocks,date,fair_value)

#analysis from https://www.morningstar.com/articles/973345/5-more-stocks-we-like
#buy as fair value > current price
stocks = ['CVE','BUD','CTVA','AXP','ECL']
date = '2020-03-19'
fair_value = [7,96,40,125,191]
morningstar(stocks, date, fair_value)

#analysis from https://www.morningstar.com/articles/976067/6-more-stocks-we-like (stock name 'CTL' changed to 'LUMN')
#buy as fair value > current price
stocks = ['CLR', 'LUMN', 'VVV', 'INGR', 'TWNK', 'GOOG']
date = '2020-04-02'
fair_value = [19,18,19.8,125,15.8,1400]
stock_dict = {'stocks':['CLR', 'LUMN', 'VVV', 'INGR', 'TWNK', 'GOOG'], 'date':'2020-04-02', 
              'fair_value':[19,18,19.8,125,15.8,1400]}
morningstar(stocks, date, fair_value)

#analysis from https://www.morningstar.com/articles/999855/3-expensive-diy-stocks-to-watch
# sell as fair value < current price
stocks = ['HD', 'LOW', 'SHW']
date = '2020-08-28'
fair_value = [200, 133, 360]
stock_dict = {'stocks':['HD', 'LOW', 'SHW'], 'date':'2020-08-28', 'fair_value':[200, 133, 360]}
morningstar(stocks, date, fair_value)

fairvalue_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Stock,Current date,Current price,Fair value estimate,Actual price,Time to reach fair/max/min value,Profit,Hit fair value?
0,FANG,2020-03-17,$20.23,$90,$85.27,"[2021-03-05, 353 days 00:00:00]",322%,No
1,FSLR,2020-03-17,$32.26,$59,$61.78,"[2020-07-14, 119 days 00:00:00]",92%,Yes
2,SYY,2020-03-17,$35.06,$59,$59.1,"[2020-06-03, 78 days 00:00:00]",69%,Yes
3,CMCSA,2020-03-17,$38.22,$49,$49.0,"[2020-11-13, 241 days 00:00:00]",28%,Yes
4,CVE,2020-03-19,$1.64,$7,$7.19,"[2021-02-22, 340 days 00:00:00]",338%,Yes
5,BUD,2020-03-19,$36.36,$96,$72.14,"[2021-04-29, 406 days 00:00:00]",98%,No
6,CTVA,2020-03-19,$23.26,$40,$41.07,"[2021-01-06, 293 days 00:00:00]",77%,Yes
7,AXP,2020-03-19,$77.07,$125,$125.04,"[2020-12-04, 260 days 00:00:00]",62%,Yes
8,ECL,2020-03-19,$159.58,$191,$197.58,"[2020-04-28, 40 days 00:00:00]",24%,Yes
9,CLR,2020-04-02,$8.34,$19,$21.58,"[2020-06-08, 67 days 00:00:00]",159%,Yes


In [384]:
fairvalue('HD', 200, '2020-08-27' )

[*********************100%***********************]  1 of 1 completed
HD
current date: 2020-08-27
current price: $288.63
fair value estimate: $200
Highest price: $328.08 on 2021-04-16 00:00:00
lowest price did not hit fair value: $250.93 on 2021-03-04 00:00:00


## yfinance recommendations
The package yfinance allows users to download historical market data from yahoo finance. It also provides recommendations which will be validated. 

In [3]:
uber_df = yf.Ticker('UBER')
uber_df = uber_df.recommendations.reset_index()
#data cleaning to get data from Jan 2020 - Apr 2021
uber_df['Date'] = pd.to_datetime(uber_df['Date']).dt.date #get rid of the time and keep the date portion 
for i in range(len(uber_df)):
    if uber_df['Date'][i].year==2020:
        print(i)
        break
for i in range(len(uber_df)):
    if uber_df['Date'][i].year==2021 and uber_df['Date'][i].month==5:
        print(i)
        break
uber_df = uber_df.iloc[47:119]

47
119


In [4]:
#Example from Uber
uber_df.head()

,Date,Firm,To Grade,From Grade,Action
47,2020-01-10,Bernstein,Outperform,,init
48,2020-01-23,Guggenheim,Buy,,main
49,2020-01-28,UBS,Buy,,init
50,2020-01-31,JP Morgan,Overweight,,init
51,2020-02-03,Wedbush,Outperform,,main


In [181]:
print(uber_df['Firm'].unique())
print('\n')
print(uber_df['To Grade'].unique())
print('\n')
print(uber_df['From Grade'].unique())
print('\n')
print(uber_df['Action'].unique())

#'Firm' are either banks or investment firms that provide their analyses on the stock.
#'To Grade' are their updated analyses, 'From Grade' are their previous analyses. 
#'Outperform, buy, overweight, market outperform' suggests that future prices will rise and recommendation is to buy the stock.
#'Neutral, equal-weight, peer perform' suggests that stock will perform similarly to expected market returns,  
# and recommendation is to neither buy nor sell. 

['Bernstein' 'Guggenheim' 'UBS' 'JP Morgan' 'Wedbush' 'Morgan Stanley'
 'Mizuho' 'RBC Capital' 'Stifel' 'MKM Partners' 'Canaccord Genuity'
 'DA Davidson' 'CFRA' 'Needham' 'Oppenheimer' 'KeyBanc' 'Wells Fargo'
 'Deutsche Bank' 'B of A Securities' 'Raymond James' 'Wolfe Research'
 'BTIG' 'JMP Securities' 'Barclays' 'Daiwa Capital' 'Citigroup'
 'Vertical Research' 'Jefferies' 'Nomura']


['Outperform' 'Buy' 'Overweight' 'Neutral' 'Market Outperform']


['' 'Neutral' 'Equal-Weight' 'Peer Perform' 'Outperform']


['init' 'main' 'up' 'reit']


In [194]:
#'To Grade' column: ['Outperform' 'Buy' 'Overweight' 'Neutral' 'Market Outperform']
#Test for several timeframes (3mth,6mth,9mth,12mth), whether price increases since recommendation is to buy
counter = 0
for i in range(len(uber_df)):
    if uber_df['To Grade'].iloc[i] == 'Neutral':
        counter += 1
print('Number of Neutral recommendations: ' + str(counter))


Number of Neutral recommendations: 1


In [204]:
delta_df = yf.Ticker('DAL')
delta_df = delta_df.recommendations.reset_index()
delta_df['Date'] = pd.to_datetime(delta_df['Date']).dt.date #get rid of the time and keep the date portion 
for i in range(len(delta_df)):
    if delta_df['Date'][i].year==2020:
        print(i)
        break
for i in range(len(delta_df)):
    if delta_df['Date'][i].year==2021 and delta_df['Date'][i].month==4:
        print(i)
        break
#delta_df = delta_df.iloc[140:172]
delta_df['To Grade'].unique()

140
172


array(['Outperform', 'Buy', 'Sell', 'Hold', 'Overweight', 'Underweight',
       'Market Perform', 'Equal-Weight', 'Neutral', 'Strong Buy',
       'Peer Perform', 'In-Line', 'Underperform'], dtype=object)

In [ ]:
#which firms recommend buy and which recommend sell/neutral? 